<a href="https://colab.research.google.com/github/DhawaDong/Tea-Leaf-Disease-Detection/blob/main/CNN_TeaLeafDiseaseDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import time
import csv
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import random
from PIL import Image
import torch.nn as nn
import albumentations
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader #TensorDataset
from torchvision import transforms
from torchvision.datasets import ImageFolder
from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [ ]:
''' SEED Everything '''
def seed_everything(SEED=42):
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.benchmark = True
SEED=42
seed_everything(SEED=SEED)
''' SEED Everything '''
# set computation device
device = ('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Computation device: {device}")

In [ ]:
#function to create csv file of image path and corresponding image class
def create_image_csv(root_folder, csv_filename):
    with open(csv_filename, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['Class', 'Image_Path'])  # Header

        for class_name in os.listdir(root_folder):
            class_folder = os.path.join(root_folder, class_name)

            if os.path.isdir(class_folder):
                for image_name in os.listdir(class_folder):
                    if image_name.lower().endswith(('.jpg', '.png', '.jpeg')):
                        image_path = os.path.join(class_folder, image_name)
                        csv_writer.writerow([class_name, image_path])

In [ ]:
root_folder_path = '/content/drive/MyDrive/7#MachineLearning#Projects/tea sickness dataset'
csv_file_path = 'TeaLeaf_ImageDataSet.csv'

# Call the function to create the CSV file
create_image_csv(root_folder_path, csv_file_path)

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# Display the DataFrame
#print(df)

#save the csv file to google drive ..
#df.to_csv('/content/drive/MyDrive/7#MachineLearning#Projects/TeaLeaf_ImageDataSet.csv') # + csv_file_path)

In [ ]:
LeafDataSet = pd.read_csv('/content/drive/MyDrive/7#MachineLearning#Projects/TeaLeaf_ImageDataSet.csv');
#Imageinput ant classes....
LeafDataSet["Class"].replace({"white spot": 0, "Anthracnose": 1,
                              "healthy": 2,   "brown blight": 3,
                              "bird eye spot": 4, "algal leaf": 5,
                              "red leaf spot": 6, "gray light": 7}, inplace = True)

In [ ]:
#randomaize the dataset
Rand_LeafDataSet = LeafDataSet.sample(frac=1, random_state=46)

In [ ]:
X = Rand_LeafDataSet.Image_Path.values   #Class', 'Image_Path'
y = Rand_LeafDataSet.Class.values

In [ ]:
(xtrain, xtest, ytrain, ytest) = train_test_split(X, y, test_size=0.30, random_state=42)

In [ ]:
print(ytrain.size)

In [ ]:
#frequency of class in TrainSet
(unique, counts) = np.unique(ytrain, return_counts=True)
Trainfrequencies = np.asarray((unique, counts)).T
#Trainfrequencies

In [ ]:
# image dataset module
class DDSMimageDataset(Dataset):
    def __init__(self, path, labels, tfms=None):
        self.X = path
        self.y = labels
        # apply augmentations
        if tfms == 0:  # if validating
            self.aug = albumentations.Compose([
                albumentations.Resize(224, 224, always_apply=True),
                albumentations.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], always_apply=True)
            ])
        else:  # if training
            self.aug = albumentations.Compose([
                albumentations.Resize(224, 224, always_apply=True),
                albumentations.HorizontalFlip(p=0.5),  # Adjust probability as needed
                albumentations.ShiftScaleRotate(shift_limit=0.3, scale_limit=0.3, rotate_limit=30, p=0.5),  # Adjust parameters as needed
                albumentations.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], always_apply=True)
            ])

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        image = Image.open(self.X[i])
        image = self.aug(image=np.array(image))['image']
        label = self.y[i]
        # Transpose the image to have channels first (C, H, W)
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        return torch.tensor(image, dtype=torch.float), torch.tensor(label, dtype=torch.long)


In [ ]:
BatchSize = 64
#trinaing data_set
train_data = DDSMimageDataset(xtrain, ytrain, tfms=1)
train_loader = DataLoader(train_data, batch_size=BatchSize, shuffle=True)

#testing data_set
test_data = DDSMimageDataset(xtest, ytest, tfms=0)
test_loader = DataLoader(test_data, batch_size=BatchSize, shuffle=False)

In [ ]:
# Define CNN architecture
class TeaLeafClassifier(nn.Module):
    def __init__(self, num_classes=8):  # Specify the number of classes
        super(TeaLeafClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        #self.fc1 = nn.Linear(32 * 16 * 16, 256)
        self.fc1 = nn.Linear(32 * 56 * 56, 256)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(256, num_classes)  # Adjust the number of output neurons
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.conv1(x)
        #print("After conv1:", x.size())
        x = self.relu1(x)
        x = self.pool1(x)

        x = self.conv2(x)
        #print("After conv2:", x.size())
        x = self.relu2(x)
        x = self.pool2(x)

        #x = x.view(-1, 32 * 16 * 16)
        x = x.view(-1, 32 * 56 * 56)  # Reshape for fully connected layer
        #print("After flattening:", x.size())

        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        #x = self.softmax(x)

        return x

In [ ]:
# Instantiate the model
model = TeaLeafClassifier(num_classes=8).to(device)

In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images.to(device))
        #loss = criterion(outputs, torch.argmax(labels.to(device), dim=1))
        loss = criterion(outputs, labels.to(device))
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

In [ ]:
# Evaluate the model on the test set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images.to(device))
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels.to(device)).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {accuracy:.4f}')